# **시계열 데이터 처리**

## **1. 환경 준비**

In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**시계열 데이터란?**
- 행과 행에 시간의 순서(흐름)가 있고
- 행과 행의 시간간격이 동일한 데이터
- Time Series

In [34]:
sales = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_sales_simple.csv")
products = pd.read_csv("https://raw.githubusercontent.com/DA4BAM/dataset/master/ts_product_master.csv")

In [35]:
# 판매액 계산하기
temp = pd.merge(sales, products)
temp['Amt' ] = temp['Qty'] * temp['Price']
temp['Amt'] = (temp['Amt']/1000).round()  # 단위 1000달러

# 집계
data1 = temp.groupby(['Date', 'Category'], as_index = False)['Amt'].sum()
data2 = temp.groupby(['Date'], as_index = False)['Amt'].sum()

# 피봇
data11 = data1.pivot(index = 'Date', columns = 'Category', values = 'Amt').reset_index()

# 합치기
data = pd.merge(data2, data11)

In [36]:
data.head()

,Date,Amt,Drink,Food,Grocery,Household Goods
0,2013-01-01,20.0,7.0,4.0,6.0,3.0
1,2013-01-02,3938.0,604.0,549.0,1663.0,1122.0
2,2013-01-03,2885.0,444.0,376.0,1222.0,843.0
3,2013-01-04,2907.0,490.0,386.0,1252.0,779.0
4,2013-01-05,3831.0,704.0,505.0,1560.0,1062.0


### 2. 날짜 요소 추출하기
**(1)날짜 타입으로 변환**
- pd.to_datetime(날짜 데이터, format = '입력되는 날짜 형식')

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             31 non-null     object 
 1   Amt              31 non-null     float64
 2   Drink            31 non-null     float64
 3   Food             31 non-null     float64
 4   Grocery          31 non-null     float64
 5   Household Goods  31 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.6+ KB


In [38]:
data['Date'] = pd.to_datetime(data['Date'])

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             31 non-null     datetime64[ns]
 1   Amt              31 non-null     float64       
 2   Drink            31 non-null     float64       
 3   Food             31 non-null     float64       
 4   Grocery          31 non-null     float64       
 5   Household Goods  31 non-null     float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 1.6 KB


- format = ''

    * pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션

In [40]:
# pd.to_datetime(date, format = '%d/%m/%Y') # format = '%d/%m/%Y' 입력되는 날짜가 이런 형태야~~ 라고 알려주는 옵션
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date)
date

0   2023-03-01
1   2023-03-02
2   2023-03-03
dtype: datetime64[ns]

In [41]:
# 만약 dd-mm-yyyy 이 맞다면!
date = pd.Series(['03-01-2023', '03-02-2023', '03-03-2023'])
date = pd.to_datetime(date, format = '%d-%m-%Y') #입력받은 날짜 데이터 형식이 '%d-%m-%Y'!
date

0   2023-01-03
1   2023-02-03
2   2023-03-03
dtype: datetime64[ns]

In [42]:
# 연도
date.dt.year

0    2023
1    2023
2    2023
dtype: int32

In [43]:
# 월
date.dt.month

0    1
1    2
2    3
dtype: int32

In [44]:
# 일
date.dt.day

0    3
1    3
2    3
dtype: int32

In [45]:
# 요일
date.dt.weekday
# 0 - 월요일 ~ 6 - 일요일

0    1
1    4
2    4
dtype: int32

In [46]:
# 요일 이름
date.dt.day_name()

0    Tuesday
1     Friday
2     Friday
dtype: object

## **3. 시간에 따른 흐름 추가하기**

### **(1) Shift**
- 시계열 데이터에서 시간의 흐름 전후로 정보를 이동시킬 때 사용

In [47]:
temp = data.loc[:,['Date','Amt']]
temp

,Date,Amt
0,2013-01-01,20.0
1,2013-01-02,3938.0
2,2013-01-03,2885.0
3,2013-01-04,2907.0
4,2013-01-05,3831.0
5,2013-01-06,4066.0
6,2013-01-07,2700.0
7,2013-01-08,2533.0
8,2013-01-09,2462.0
9,2013-01-10,2110.0


In [48]:
# 전날 매출액 열을 추가
temp['Amt_lag'] = temp['Amt'].shift() # default = 1
# 전전날 매출액 열을 추가.
temp['Amt_lag2'] = temp['Amt'].shift(2) # 2행 shift
# 다음날 매출액 열을 추가
temp['Amt_lag_1'] = temp['Amt'].shift(-1)

temp.head()

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1
0,2013-01-01,20.0,NaN,NaN,3938.0
1,2013-01-02,3938.0,20.0,NaN,2885.0
2,2013-01-03,2885.0,3938.0,20.0,2907.0
3,2013-01-04,2907.0,2885.0,3938.0,3831.0
4,2013-01-05,3831.0,2907.0,2885.0,4066.0


### **(2) rolling + 집계함수**
- 시간의 흐름에 따라 일정 기간 동안 평균을 이동하면서 구하기
- .rolling(n)
    * n의 deflaut 값은 1
    * min_periods: 최소 데이터 수

In [49]:
# 7일 이동평균 매출액을 구해 봅시다.
temp['Amt_MA7_1'] = temp['Amt'].rolling(7).mean()
temp['Amt_MA7_2'] = temp['Amt'].rolling(7, min_periods = 1).mean()
temp.head(10)

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1,Amt_MA7_1,Amt_MA7_2
0,2013-01-01,20.0,NaN,NaN,3938.0,NaN,20.000000
1,2013-01-02,3938.0,20.0,NaN,2885.0,NaN,1979.000000
2,2013-01-03,2885.0,3938.0,20.0,2907.0,NaN,2281.000000
3,2013-01-04,2907.0,2885.0,3938.0,3831.0,NaN,2437.500000
4,2013-01-05,3831.0,2907.0,2885.0,4066.0,NaN,2716.200000
5,2013-01-06,4066.0,3831.0,2907.0,2700.0,NaN,2941.166667
6,2013-01-07,2700.0,4066.0,3831.0,2533.0,2906.714286,2906.714286
7,2013-01-08,2533.0,2700.0,4066.0,2462.0,3265.714286,3265.714286
8,2013-01-09,2462.0,2533.0,2700.0,2110.0,3054.857143,3054.857143
9,2013-01-10,2110.0,2462.0,2533.0,2405.0,2944.142857,2944.142857


### (3) **diff**
* 특정 시점 데이터, 이전시점 데이터와의 차이 구하기

In [50]:
temp['Amt_D1'] = temp['Amt'].diff()
temp['Amt_D2'] = temp['Amt'].diff(2)
temp.head(10)

,Date,Amt,Amt_lag,Amt_lag2,Amt_lag_1,Amt_MA7_1,Amt_MA7_2,Amt_D1,Amt_D2
0,2013-01-01,20.0,NaN,NaN,3938.0,NaN,20.000000,NaN,NaN
1,2013-01-02,3938.0,20.0,NaN,2885.0,NaN,1979.000000,3918.0,NaN
2,2013-01-03,2885.0,3938.0,20.0,2907.0,NaN,2281.000000,-1053.0,2865.0
3,2013-01-04,2907.0,2885.0,3938.0,3831.0,NaN,2437.500000,22.0,-1031.0
4,2013-01-05,3831.0,2907.0,2885.0,4066.0,NaN,2716.200000,924.0,946.0
5,2013-01-06,4066.0,3831.0,2907.0,2700.0,NaN,2941.166667,235.0,1159.0
6,2013-01-07,2700.0,4066.0,3831.0,2533.0,2906.714286,2906.714286,-1366.0,-1131.0
7,2013-01-08,2533.0,2700.0,4066.0,2462.0,3265.714286,3265.714286,-167.0,-1533.0
8,2013-01-09,2462.0,2533.0,2700.0,2110.0,3054.857143,3054.857143,-71.0,-238.0
9,2013-01-10,2110.0,2462.0,2533.0,2405.0,2944.142857,2944.142857,-352.0,-423.0


In [51]:
temp = data.loc[:, ['Date', 'Grocery']]
temp.head()

,Date,Grocery
0,2013-01-01,6.0
1,2013-01-02,1663.0
2,2013-01-03,1222.0
3,2013-01-04,1252.0
4,2013-01-05,1560.0


In [52]:
# 전날 매출액
temp['Gro_lag'] = temp['Grocery'].shift()

# 7일 전(전주 동 요일) 매출액
temp['Gro_lag7'] = temp['Grocery'].shift(7)

# 3일 이동평균 매출액
temp['Gro_MA3'] = temp['Grocery'].rolling(3, min_periods = 1).mean()

# 전날대비 매출액 증감여부 (증가 1, 감소 -1, 동일 0)
temp['Gro_Diff'] = np.where(temp['Grocery'].diff() > 0, 1,
                            np.where(temp['Grocery'].diff() == 0, 0, -1))

temp['Gro_Diff'] = temp['Grocery'].diff()

temp['Gro_Diff2'] = pd.cut(temp['Gro_Diff'], bins = [-np.inf, -0.5, 0.5, np.inf],
                          labels = [-1, 0, 1])

In [53]:
temp

,Date,Grocery,Gro_lag,Gro_lag7,Gro_MA3,Gro_Diff,Gro_Diff2
0,2013-01-01,6.0,NaN,NaN,6.000000,NaN,NaN
1,2013-01-02,1663.0,6.0,NaN,834.500000,1657.0,1
2,2013-01-03,1222.0,1663.0,NaN,963.666667,-441.0,-1
3,2013-01-04,1252.0,1222.0,NaN,1379.000000,30.0,1
4,2013-01-05,1560.0,1252.0,NaN,1344.666667,308.0,1
5,2013-01-06,1713.0,1560.0,NaN,1508.333333,153.0,1
6,2013-01-07,1170.0,1713.0,NaN,1481.000000,-543.0,-1
7,2013-01-08,1144.0,1170.0,6.0,1342.333333,-26.0,-1
8,2013-01-09,1074.0,1144.0,1663.0,1129.333333,-70.0,-1
9,2013-01-10,878.0,1074.0,1222.0,1032.000000,-196.0,-1
